In [5]:
import pandas as pd 
import json 
import shapefile
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os 
import json
import numpy as np

def isNested(entry): 
    try: 
        sum(entry)
        return 0 
    except: 
        return 1

def readInJson(fileName):
    try: 
        df = pd.read_json(fileName)
    except: 
        handle = json.load(open(fileName))
        df = pd.DataFrame(handle['features'])
        
    return df

lonlatSuburb = pd.read_csv('lonlatSuburb.csv')
vicPostcodes = pd.read_csv('vicPostcodes.csv')

lonlatSuburb['lat'] = lonlatSuburb.lat.apply(lambda x: round(x, 6))
lonlatSuburb['long'] = lonlatSuburb.long.apply(lambda x: round(x, 6))

vicPostcodes['lat'] = vicPostcodes.lat.apply(lambda x: round(x, 6))
vicPostcodes['long'] = vicPostcodes.long.apply(lambda x: round(x, 6))

In [6]:
garbageCollectionDate = sorted([i for i in os.listdir() if i.endswith('json')])
garbageCollectionDate[5]

'GreaterSheppartonCityCouncil-garbage-collection-zones.json'

In [7]:
readInJson(garbageCollectionDate[5]).head(2)

,type,id,geometry,geometry_name,properties
0,Feature,ckan_667482ba_9812_42c9_9471_2df96ffce136.1,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'rub_start': '2017-01-26', 'rub_day': 'Thursd..."
1,Feature,ckan_667482ba_9812_42c9_9471_2df96ffce136.2,"{'type': 'MultiPolygon', 'coordinates': [[[[14...",geom,"{'rub_start': '2017-01-30', 'rub_day': 'Monday..."


In [8]:
df = readInJson(garbageCollectionDate[5])

df['geo'] = df.geometry.apply(lambda x: x['coordinates'][0][0])

df = df.explode('geo')

df['geo_nested'] = df.geo.apply(lambda x: isNested(x))
df['lat'] = df.geo.apply(lambda x: round(x[0], 6))
df['long'] = df.geo.apply(lambda x: round(x[1], 6))

df.drop(['geo','id', 'geometry', 'geometry_name',
         'geo_nested', 'type'], axis = 1, inplace = True)

#df.drop(['geo', 'features', 'type', 'name', 'geo_nested'], axis = 1, inplace = True)

In [10]:
df.iloc[0,0]

{'rub_start': '2017-01-26',
 'rub_day': 'Thursday',
 'rub_weeks': '1',
 'rec_start': '2017-01-26',
 'rec_day': 'Thursday',
 'rec_weeks': '2',
 'grn_start': '2017-02-02',
 'grn_day': 'Thursday',
 'grn_weeks': '2',
 'name': 'Thursday B',
 'rub_name': 'red lid',
 'rec_name': 'yellow lid',
 'grn_name': 'green lid',
 'info_url': 'http://greatershepparton.com.au/animals-environment-and-waste/waste-and-recycling/kerbside-collection',
 'missed_ph': '0358329700'}

In [11]:
def collectionFrequency(row, wasteType): 
    
    rub_colDay = row[f'{wasteType}_day']
    rub_colFreq = row[f'{wasteType}_weeks']
    
    if rub_colFreq == '2' or rub_colFreq == 2: 
        rub_colFreq = 'Every Second '
    else: 
        rub_colFreq = 'Every '
    
    return rub_colFreq + rub_colDay

df['landfill_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rub'))
df['recyclable_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'rec'))
df['green_colDay'] = df.properties.apply(lambda x: collectionFrequency(x, 'grn'))

df.drop('properties', axis = 1, inplace = True)

In [13]:
shepparton = lonlatSuburb.merge(df)[['locality', 'long', 'lat', 'landfill_colDay', 'recyclable_colDay', 'green_colDay']]\
.sort_values(['locality', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
        .drop_duplicates(['locality','recyclable_colDay', 'recyclable_colDay', 'green_colDay'])\
        .sort_values(['locality'])\
.merge(vicPostcodes[['locality', 'postcode']])\
.sort_values(['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.drop_duplicates(subset = ['postcode', 'landfill_colDay', 'recyclable_colDay', 'green_colDay'])\
.rename(columns = dict(zip(['landfill_colDay', 'recyclable_colDay', 'green_colDay'], 
                           ['landfill', 'recyclable', 'organics'])))\
.drop(['long', 'lat', 'locality'], axis = 1)\
.melt(id_vars = 'postcode')\
.rename(columns = {'variable':'binInfo_bin_type', 'value':'collection_time'})\
.drop_duplicates(['postcode', 'binInfo_bin_type'])

shepparton.to_csv('../GBD_shepparton.csv', index = False)